# Part 4: Working with Files

## Finde die passende Pizza

Task: Im Folgenden Szenario wird die Speisekarte von einer exteren Agentur erstellt und ist nur als PDF verfügbar. Im folgenden passen wir unseren digitalen Kellner so an, dass er mit der PDF Speisekarte arbeitet. 

## Setup

In [ ]:
# Install required packages
%pip install openai matplotlib scikit-learn umap-learn plotly faiss-cpu numpy tabulate pandas

In [ ]:
# Import necessary libraries
from openai import OpenAI
import json
import faiss
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict
import io

# Download Menu
import urllib.request
import os.path
MENU_URL = "https://raw.githubusercontent.com/jank-bcxp/bcxp_weekend2025_HandsOn_AI/refs/heads/main/menu.py"
urllib.request.urlretrieve(MENU_URL, os.path.basename(MENU_URL))
SPEISEKARTE_URL = "https://raw.githubusercontent.com/jank-bcxp/bcxp_weekend2025_HandsOn_AI/refs/heads/main/speisekarte.pdf"
urllib.request.urlretrieve(SPEISEKARTE_URL, os.path.basename(SPEISEKARTE_URL))
WOCHENKARTE_URL = "https://raw.githubusercontent.com/jank-bcxp/bcxp_weekend2025_HandsOn_AI/refs/heads/main/wochenkarte.pdf"
urllib.request.urlretrieve(WOCHENKARTE_URL, os.path.basename(WOCHENKARTE_URL))

from menu import MENU

# Initialize OpenAI client
from google.colab import userdata
openai_client = OpenAI(api_key= userdata.get('openai_api_key'))

## Verwendung von OpenAI API mit **Structured Outputs** und **Files** Input:

Multimodale Modelle sind KI-Modelle, die Informationen aus verschiedenen Modalitäten – etwa Text, Bilder, Audio oder strukturierte Daten – gleichzeitig verarbeiten und kombinieren können. Das unterscheidet sie von klassischen Modellen, die nur mit einer einzigen Eingabemodalität (z. B. nur Text) umgehen können.

GPT 4.1. beispielsweise kann multimodalen Input wie Text, PDFs oder Bilder verarbeiten. Im folgenden Beispiel verwenden wir `gpt-4.1` zur Verarbeitung eines Nutzer-Inputs *und* einer PDF-Datei (z. B. einer Speisekarte).

**Aufgabe:** Neben Speisen für die Pizzeria soll der Kellner auch Speisen für das Restaurant "Naherholungsgebiet" in Stuttgart aufnehmen können. Erweitere den API Aufruf, sodass beide Speisekarten übergeben werden. Füge im Systemprompt den aktuellen Tag hinzu und mache Anpassungen, dass nur die für den Tag verfügbaren Speisen des Naherholungsgebiets berücksichtigt werden. Erweitere das Output-Schema so, dass auch das entsprechende Restaurant ("Naherholungsgebite" oder "Salve") in der Ausgabe enthalten sind. Finde Bestellungen für die Speisen aus beiden Restaurants empfohlen werden. Passe den Prompt wo nötig an. 

In [ ]:
import base64

user_input = "Ich habe Lust auf was mit Gorgonzola."

with open("speisekarte.pdf", "rb") as f:
    data = f.read()
speisekarte_base64_string = base64.b64encode(data).decode("utf-8")

menu_items_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

# 🧠 API-Aufruf
response = openai_client.responses.create(
    model="gpt-4.1",
    input=[
        {
            "role": "developer",
            "content": [
                {
                    "type": "input_text",
                    "text": "Du bist ein digitaler Kellner. Wähle passende Einträge aus der Speisekarte.",
                },
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "filename": "speisekarte.pdf",
                    "file_data": f"data:application/pdf;base64,{speisekarte_base64_string}",
                },
                {
                    "type": "input_text",
                    "text": f"{user_input}",
                },
            ],
        }
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": menu_items_schema,
            "strict": True,
        }
    },
)

recommendations = json.loads(response.output_text)["items"]

# 📤 Ausgabe anzeigen
for item in recommendations:
    print(item)